In [25]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from http_request_randomizer.requests.proxy.requestProxy import RequestProxy

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote

import dateparser as dp
import datetime

from PIL import Image
from io import BytesIO
import re

commons_site = pb.Site("commons", "commons")

In [26]:
from modules.wmtools import is_commons_file, get_hash

In [27]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [28]:
req_proxy = RequestProxy()

2018-01-20 21:15:44,597 root   DEBUG    === Initialized Proxy Parsers ===
2018-01-20 21:15:44,598 root   DEBUG    	 FreeProxy Parser of 'http://free-proxy-list.net' with required bandwidth: '150' KBs
2018-01-20 21:15:44,600 root   DEBUG    	 ProxyForEU Parser of 'http://proxyfor.eu/geo.php' with required bandwidth: '1.0' KBs
2018-01-20 21:15:44,601 root   DEBUG    	 RebroWeebly Parser of 'http://rebro.weebly.com' with required bandwidth: '150' KBs
2018-01-20 21:15:44,602 root   DEBUG    	 SemairProxy Parser of 'https://premproxy.com/list/' with required bandwidth: '150' KBs
2018-01-20 21:15:44,605 root   DEBUG    =================================
2018-01-20 21:15:45,187 http_request_randomizer.requests.parsers.FreeProxyParser DEBUG    Proxy Invalid: <zip object at 0x7ff84bfed048>
2018-01-20 21:15:45,539 http_request_randomizer.requests.parsers.ProxyForEuParser DEBUG    Proxy with low bandwidth: 0.178
2018-01-20 21:15:45,541 http_request_randomizer.requests.parsers.ProxyForEuParser DEBU

In [29]:
urls = ['http://www.lamoncloa.gob.es/multimedia/fototeca/Paginas/Imagen.aspx?img=085c50bb1c']
categories = ['José María Aznar in 2001',
              'Historians from Spain']

In [30]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Fototeca: ${title}]
|author=Ministerio de la Presidencia. Gobierno de España (${author})
|permission=
|other versions=
}}

=={{int:license-header}}==
{{LicenseReview}}
{{attribution-La Moncloa}}

[[Category:Files uploaded by User:Discasto]]
${cat_string}"""

In [31]:
used_names = []

for url in urls :
    cats = categories[:]
    while True:
        request = req_proxy.generate_proxied_request(url)
        if request is not None:
            #print("{0}".format(u''.join(request.text).encode('utf-8')))
            response = request.text
            break
        else :
            print ('Trying')

    soup = BeautifulSoup(response, 'html.parser')
    author = contents[0]
    if "Pool" not in author :
        print ("Image from a party other than Moncloa. Skipping")
        continue        

    title = soup.find_all("h1", class_="contenidoTitulo")[0].get_text().strip().replace(':', ' - ').replace('  ', ' ')
    try:
        date_string = soup.find_all("div", class_="contenidoFecha")[0].get_text()
        pub_date_sp = ' '.join(date_string.strip().split(' ')[-4:])
        parsed_pub_date = dp.parse(pub_date_sp)
        pub_date = parsed_pub_date.strftime('%Y-%m-%d')
    
        year_date = parsed_pub_date.strftime('%Y')
        if 'La Moncloa' in date_string:
            cats.append('Palacio de la Moncloa')
            cats.append(year_date + ' in Madrid')
    except :
        pub_date = None
    contents = soup.find_all("div", class_="entradillaContenedor")[0].get_text().strip().replace('\t', '').split('\r\n')
    contents = [item.replace('\n', '') for item in contents if len(item)>0]

    description = contents[1]
    image_url = 'http://www.lamoncloa.gob.es' + soup.find_all("a", id="ctl00_PlaceHolderMain_DisplayMode_aDescargar")[0]['href']

    vars = {
        "url": image_url,
        "description": description,
        "author": author,
        "date": pub_date,
        "title": title,
        "cat_string": '\n'.join(['[[Category:'+i+']]' for i in cats])
    }
    t = Template(template)
    _text = t.render(**vars)
    
    # First, the image is downloaded and stored
    image_name = '{}.jpg'.format(title.replace(':', ' -').replace('  ', ' '))
    image_path = os.path.join(images_directory, image_name)

    while True:
        print ('Trying download')
        img_request = req_proxy.generate_proxied_request(image_url)
        if img_request is not None:
            image = Image.open(BytesIO(img_request.content))
            image.save(image_path)
            print ('Image accessed')
            break
        else :
            print ('Still trying download')

    # If the image is already in Commons, skip
    if is_commons_file(get_hash(image_path)) :
        print ("Image already in commons. Leaving")
    else :
        # If the image name is already in commons, find a new name
        if pb.Page(commons_site, image_name, ns=6).exists():
            print ("Image name ({0}) already used in Commons".format(image_name))
            used_names.append(image_name)
        
        while True:
            if image_name in used_names :
                # Finding a new name
                image_subject = '.'.join(image_name.split('.')[:-1])
                image_extension = 'jpg'
                p = re.compile('(.*) ([0-9]{2}\.jpg)')
                m = p.match(image_name)
                if m is None:
                    image_name = image_subject + ' 01.' + image_extension
                else :
                    counter = int(m.group(2)[:2]) + 1
                    image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

                if pb.Page(commons_site, image_name, ns=6).exists():
                    print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                    used_names.append(image_name)
            else :
                print ("Preparing to upload image with name {0}".format(image_name))
                used_names.append(image_name)
                break

        # image upload
        bot = UploadRobot(image_path,
                          description = _text,
                          useFilename = image_name,
                          keepFilename = True,
                          verifyDescription = False,
                          ignoreWarning = True,
                          targetSite = commons_site)
        #bot.run()

    os.remove(image_path)

2018-01-20 21:15:55,648 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; es-ES; rv:1.9.1) Gecko/20090624 Firefox/3.5 (.NET CLR 3.5.30729'}
2018-01-20 21:15:55,650 root   DEBUG    Using proxy: 61.247.186.137:8080 | Samair
2018-01-20 21:16:02,696 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; de; rv:1.9.1.1) Gecko/20090715 Firefox/3.5.'}
2018-01-20 21:16:02,697 root   DEBUG    Using proxy: 113.53.61.80:8080 | FreeProxy


RR Status 200
Trying download


2018-01-20 21:16:32,726 root   DEBUG    Proxy unreachable - Removed Straggling proxy: 113.53.61.80:8080 | FreeProxy PL Size = 1492
2018-01-20 21:16:32,731 root   DEBUG    Using headers: {'Connection': 'close', 'User-Agent': 'Mozilla/5.0 (iPad; U; CPU iPhone OS 3_2 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B31'}
2018-01-20 21:16:32,733 root   DEBUG    Using proxy: 180.183.98.186:8080 | FreeProxy


Still trying download
Trying download


2018-01-20 21:16:53,326 pywiki VERBOSE  Found 1 commons:commons processes running, including this one.
2018-01-20 21:16:53,331 requests_oauthlib.oauth1_auth DEBUG    Signing request <PreparedRequest [POST]> using client <Client client_key=0a73e346a40b07262b6e36bdba01cba4, client_secret=****, resource_owner_key=5ca4fafa5dd85bb0ecf516d96ad6bf07, resource_owner_secret=****, signature_method=HMAC-SHA1, signature_type=AUTH_HEADER, callback_uri=None, rsa_key=None, verifier=None, realm=None, encoding=utf-8, decoding=utf-8, nonce=None, timestamp=None>
2018-01-20 21:16:53,333 requests_oauthlib.oauth1_auth DEBUG    Including body in call to sign: True
2018-01-20 21:16:53,336 oauthlib.oauth1.rfc5849 DEBUG    Collected params: [('oauth_nonce', '16917610888969901381516483013'), ('oauth_timestamp', '1516483013'), ('oauth_version', '1.0'), ('oauth_signature_method', 'HMAC-SHA1'), ('oauth_consumer_key', '0a73e346a40b07262b6e36bdba01cba4'), ('oauth_token', '5ca4fafa5dd85bb0ecf516d96ad6bf07'), ('gaifrom

RR Status 200
Image accessed


2018-01-20 21:16:53,346 oauthlib.oauth1.rfc5849 DEBUG    Normalized URI: https://commons.wikimedia.org/w/api.php
2018-01-20 21:16:53,347 oauthlib.oauth1.rfc5849 DEBUG    Signing: signature base string: POST&https%3A%2F%2Fcommons.wikimedia.org%2Fw%2Fapi.php&action%3Dquery%26continue%3D%26format%3Djson%26gaifrom%3D%2521%26gailimit%3D500%26gaisha1%3Daa3db18132f20544b7ccf7277b9b4b0cdd2def06%26generator%3Dallimages%26iiprop%3Dtimestamp%257Cuser%257Ccomment%257Curl%257Csize%257Csha1%257Cmetadata%26indexpageids%3D%26inprop%3Dprotection%26maxlag%3D5%26meta%3Duserinfo%26oauth_consumer_key%3D0a73e346a40b07262b6e36bdba01cba4%26oauth_nonce%3D16917610888969901381516483013%26oauth_signature_method%3DHMAC-SHA1%26oauth_timestamp%3D1516483013%26oauth_token%3D5ca4fafa5dd85bb0ecf516d96ad6bf07%26oauth_version%3D1.0%26prop%3Dinfo%257Cimageinfo%257Ccategoryinfo%26uiprop%3Dblockinfo%257Chasmsg
2018-01-20 21:16:53,348 oauthlib.oauth1.rfc5849 DEBUG    Signature: 9+6jeUBJAtTgj94U1pQcHq0jeYo=
2018-01-20 21:16:53

[[commons:File:José María Aznar recibe un volumen sobre "El Sitio" de Bilbao del historiador José Manuel Azcona.jpg]]
Image already in commons. Leaving
